# End to End Chicken Disease Classification Project

## Stage 3 : Model Training

In [1]:
%pwd

'd:\\Documents\\Projects\\chiicken_disease_classification\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\Documents\\Projects\\chiicken_disease_classification'

### Update entity

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

### Update configuration manager in src config

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILEPATH,
                 params_filepath=PARAMS_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        training_data = os.path.join(
            self.config.data_ingestion.unzip_dir,
            "Chicken-fecal-images"
        )

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path), updated_model_path=Path(prepare_base_model.updated_model_path),
            training_data=training_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE,
            params_is_augmentation=params.AUGMENTATION
        )

        return training_config

### Update components

In [6]:
import tensorflow as tf


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_model_path
        )

    def prepare_data_for_training(self):
        train_ds = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.2,
            subset="training",
            seed=42,
            batch_size=self.config.params_batch_size,
            image_size=(
                self.config.params_image_size[0], self.config.params_image_size[1])
        )

        val_ds = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.2,
            subset="validation",
            seed=42,
            batch_size=self.config.params_batch_size,
            image_size=(self.config.params_image_size[0], self.config.params_image_size[1]))
        
        if self.config.params_is_augmentation:
            train_aug = tf.keras.Sequential([
                        tf.keras.layers.Rescaling(scale=1.0 / 255),
                        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
                        tf.keras.layers.RandomZoom(
                            height_factor=(-0.05, -0.15),
                            width_factor=(-0.05, -0.15)),
                        tf.keras.layers.RandomRotation(0.3)
                        ])
            
            test_aug = tf.keras.Sequential([
	                    tf.keras.layers.Rescaling(scale=1.0 / 255)
                        ])
            
            self.train_ds = train_ds.map(lambda x, y: (train_aug(x), y),
		 num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
            self.val_ds = val_ds.map(lambda x, y: (test_aug(x), y),
		num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
            

        else:
            self.train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
            self.val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

    def train(self):
        self.model.fit(
            self.train_ds,
            epochs=self.config.params_epochs,
            validation_data=self.val_ds,
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

### Update pipeline

In [7]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.prepare_data_for_training()
    training.train()
except Exception as e:
    raise e

[2023-10-19 16:06:27,664:  INFO: common: yaml file: config\config.yaml loaded]
[2023-10-19 16:06:27,672:  INFO: common: yaml file: params.yaml loaded]
[2023-10-19 16:06:27,676:  INFO: common: Created directory at artifacts]
Found 390 files belonging to 2 classes.
Using 312 files for training.
Found 390 files belonging to 2 classes.
Using 78 files for validation.
20/20 [==============================] - 24s 324ms/step - loss: 77.5475 - accuracy: 0.5256 - val_loss: 1.4018 - val_accuracy: 0.5128
